# An Analysis of Cell Types found in the Primary Motor Cortex (M1) of *Homo sapiens*

This project attempts to correlate the biological functions of different M1 cell types with their phenotypic expression.

The project uses **single-nucleus RNA-seq** data, obtained from the study [Bakken et al., 2021](https://www.nature.com/articles/s41586-021-03465-8#Abs1). It is available to download from the [Allen Brain Institute](https://portal.brain-map.org/atlases-and-data/rnaseq/human-m1-10x).

## 1. Install and load the required packages.

In [9]:
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager", version = "3.21")

In [3]:
BiocManager::install(c("DESeq2", "biomaRt"), quietly = TRUE)
install.packages(c("tidyverse", "pheatmap"), quietly = TRUE)

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: http://cran.rstudio.com/

Bioconductor version 3.19 (BiocManager 1.30.23), R 4.4.0 (2024-04-24)

Installing package(s) 'DESeq2', 'biomaRt'

also installing the dependencies ‘UCSC.utils’, ‘GenomeInfoDbData’, ‘zlibbioc’, ‘SparseArray’, ‘Biostrings’, ‘GenomeInfoDb’, ‘XVector’, ‘S4Arrays’, ‘DelayedArray’, ‘KEGGREST’, ‘S4Vectors’, ‘IRanges’, ‘GenomicRanges’, ‘SummarizedExperiment’, ‘Biobase’, ‘BiocParallel’, ‘MatrixGenerics’, ‘AnnotationDbi’, ‘BiocFileCache’


Old packages: 'abind', 'actuar', 'ade4', 'adegraphics', 'adehabitatHR',
  'adehabitatLT', 'adehabitatMA', 'admisc', 'AER', 'afex', 'aglm', 'AID',
  'akc', 'ale', 'AlgDesign', 'AlphaSimR', 'amap', 'Amelia', 'anacor',
  'antiword', 'anytime', 'ape', 'aplot', 'apollo', 'aqp', 'arrow', 'ars',
  'ARTool', 'AsioHeaders', 'askpass', 'ASMap', 'astsa', 'automap', 'baguette',
  'B

In [4]:
library(DESeq2)
library(biomaRt)
library(tidyverse)
library(pheatmap)

Loading required package: S4Vectors

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, table,
    tapply, union, unique, unsplit, which.max, which.min



Attaching package: ‘S4Vectors’


The following object is masked from ‘package:utils’:

    findMatches


The following objects are masked from ‘package:base’:

    expand.grid, I, unname


Loading required package: IRanges

Loading required package: GenomicRanges

Loading required package: GenomeInfoDb

Loading r

## 2. Read in the expression matrix and the metadata file.

In [5]:
options(timeout = 1000)
matrix_link <- "https://idk-etl-prod-download-bucket.s3.amazonaws.com/aibs_human_m1_10x/matrix.csv"
gene_matrix <- data.table::fread(matrix_link)

In [6]:
meta_filepath <- "/kaggle/input/metadata/metadata.csv"
meta_data <- read.csv(meta_filepath)

## 3. Explore and clean the metadata file.

In [26]:
str(meta_data)

'data.frame':	76533 obs. of  39 variables:
 $ sample_name                : chr  "AAACCCAAGGATTTCC-LKTX_190129_01_A01" "AAACCCAAGTATGGCG-LKTX_190129_01_A01" "AAACCCACAAAGTGTA-LKTX_190129_01_A01" "AAACCCACACTACTTT-LKTX_190129_01_A01" ...
 $ exp_component_name         : chr  "AAACCCAAGGATTTCC-21L8TX_180927_001_A01" "AAACCCAAGTATGGCG-21L8TX_180927_001_A01" "AAACCCACAAAGTGTA-21L8TX_180927_001_A01" "AAACCCACACTACTTT-21L8TX_180927_001_A01" ...
 $ cluster_label              : chr  "Inh L1-2 SST CCNJL" "Exc L5-6 FEZF2 IFNG-AS1" "Exc L3-5 RORB LINC01202" "Exc L2 LINC00507 GLRA3" ...
 $ cluster_color              : chr  "#fb8d00" "#2c815f" "#547d7a" "#cecd32" ...
 $ cluster_order              : int  50 116 87 75 120 101 111 90 90 100 ...
 $ class_label                : chr  "GABAergic" "Glutamatergic" "Glutamatergic" "Glutamatergic" ...
 $ class_color                : chr  "#FF7373" "#3DCC3D" "#3DCC3D" "#3DCC3D" ...
 $ class_order                : int  1 2 2 2 3 2 2 2 2 2 ...
 $ subclass_label   

In [7]:
meta_data <- meta_data %>% select(!ends_with("_color") & !ends_with("_order"))
colnames(meta_data)

[1] "sample_name"                 "exp_component_name"         
 [3] "cluster_label"               "class_label"                
 [5] "subclass_label"              "donor_sex_label"            
 [7] "region_label"                "cortical_layer_label"       
 [9] "cell_type_accession_label"   "cell_type_alias_label"      
[11] "cell_type_alt_alias_label"   "cell_type_designation_label"
[13] "external_donor_name_label"   "specimen_type"              
[15] "full_genotype_label"         "outlier_call"               
[17] "outlier_type"

In [5]:
head(meta_data)

,sample_name,exp_component_name,cluster_label,class_label,subclass_label,donor_sex_label,region_label,cortical_layer_label,cell_type_accession_label,cell_type_alias_label,cell_type_alt_alias_label,cell_type_designation_label,external_donor_name_label,specimen_type,full_genotype_label,outlier_call,outlier_type
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>,<chr>,<lgl>
1,AAACCCAAGGATTTCC-LKTX_190129_01_A01,AAACCCAAGGATTTCC-21L8TX_180927_001_A01,Inh L1-2 SST CCNJL,GABAergic,Sst,F,M1,all,CS1912131050,Inh L1-2 SST CCNJL,,Neuron 50,H18.30.001,nucleus,NA,False,NA
2,AAACCCAAGTATGGCG-LKTX_190129_01_A01,AAACCCAAGTATGGCG-21L8TX_180927_001_A01,Exc L5-6 FEZF2 IFNG-AS1,Glutamatergic,L5/6 NP,F,M1,all,CS1912131116,Exc L5-6 FEZF2 IFNG-AS1,,Neuron 116,H18.30.001,nucleus,NA,False,NA
3,AAACCCACAAAGTGTA-LKTX_190129_01_A01,AAACCCACAAAGTGTA-21L8TX_180927_001_A01,Exc L3-5 RORB LINC01202,Glutamatergic,L5 IT,F,M1,all,CS1912131087,Exc L3-5 RORB LINC01202,,Neuron 87,H18.30.001,nucleus,NA,False,NA
4,AAACCCACACTACTTT-LKTX_190129_01_A01,AAACCCACACTACTTT-21L8TX_180927_001_A01,Exc L2 LINC00507 GLRA3,Glutamatergic,L2/3 IT,F,M1,all,CS1912131075,Exc L2 LINC00507 GLRA3,,Neuron 75,H18.30.001,nucleus,NA,False,NA
5,AAACCCACAGTGAGCA-LKTX_190129_01_A01,AAACCCACAGTGAGCA-21L8TX_180927_001_A01,Oligo L2-6 OPALIN FTH1P3,Non-Neuronal,Oligo,F,M1,all,CS1912131120,Oligo L2-6 OPALIN FTH1P3,,Non-neuron 3,H18.30.001,nucleus,NA,False,NA
6,AAACCCAGTCACCCTT-LKTX_190129_01_A01,AAACCCAGTCACCCTT-21L8TX_180927_001_A01,Exc L5-6 FEZF2 C9orf135-AS1,Glutamatergic,L6 CT,F,M1,all,CS1912131101,Exc L5-6 FEZF2 C9orf135-AS1,L6 CT_2,Neuron 101,H18.30.001,nucleus,NA,False,NA


In [7]:
# To show that these columns only contain NA values
print(sum(is.na(meta_data$outlier_type)))
print(sum(is.na(meta_data$full_genotype_label)))
print(nrow(meta_data))

[1] 76533
[1] 76533
[1] 76533


In [8]:
meta_data <- select(meta_data, -full_genotype_label, -outlier_type)
head(meta_data)

,sample_name,exp_component_name,cluster_label,class_label,subclass_label,donor_sex_label,region_label,cortical_layer_label,cell_type_accession_label,cell_type_alias_label,cell_type_alt_alias_label,cell_type_designation_label,external_donor_name_label,specimen_type,outlier_call
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,AAACCCAAGGATTTCC-LKTX_190129_01_A01,AAACCCAAGGATTTCC-21L8TX_180927_001_A01,Inh L1-2 SST CCNJL,GABAergic,Sst,F,M1,all,CS1912131050,Inh L1-2 SST CCNJL,,Neuron 50,H18.30.001,nucleus,False
2,AAACCCAAGTATGGCG-LKTX_190129_01_A01,AAACCCAAGTATGGCG-21L8TX_180927_001_A01,Exc L5-6 FEZF2 IFNG-AS1,Glutamatergic,L5/6 NP,F,M1,all,CS1912131116,Exc L5-6 FEZF2 IFNG-AS1,,Neuron 116,H18.30.001,nucleus,False
3,AAACCCACAAAGTGTA-LKTX_190129_01_A01,AAACCCACAAAGTGTA-21L8TX_180927_001_A01,Exc L3-5 RORB LINC01202,Glutamatergic,L5 IT,F,M1,all,CS1912131087,Exc L3-5 RORB LINC01202,,Neuron 87,H18.30.001,nucleus,False
4,AAACCCACACTACTTT-LKTX_190129_01_A01,AAACCCACACTACTTT-21L8TX_180927_001_A01,Exc L2 LINC00507 GLRA3,Glutamatergic,L2/3 IT,F,M1,all,CS1912131075,Exc L2 LINC00507 GLRA3,,Neuron 75,H18.30.001,nucleus,False
5,AAACCCACAGTGAGCA-LKTX_190129_01_A01,AAACCCACAGTGAGCA-21L8TX_180927_001_A01,Oligo L2-6 OPALIN FTH1P3,Non-Neuronal,Oligo,F,M1,all,CS1912131120,Oligo L2-6 OPALIN FTH1P3,,Non-neuron 3,H18.30.001,nucleus,False
6,AAACCCAGTCACCCTT-LKTX_190129_01_A01,AAACCCAGTCACCCTT-21L8TX_180927_001_A01,Exc L5-6 FEZF2 C9orf135-AS1,Glutamatergic,L6 CT,F,M1,all,CS1912131101,Exc L5-6 FEZF2 C9orf135-AS1,L6 CT_2,Neuron 101,H18.30.001,nucleus,False


In [11]:
# Count the number of blank entries in the cell_type_alt_alias_label column
num_blank <- sum(meta_data$cell_type_alt_alias_label=="")
print(num_blank)

# Count the number of values in this column
print(nrow(meta_data) - num_blank)

[1] 71468
[1] 5065


In [9]:
meta_data <- select(meta_data, -cell_type_alt_alias_label)
head(meta_data)

,sample_name,exp_component_name,cluster_label,class_label,subclass_label,donor_sex_label,region_label,cortical_layer_label,cell_type_accession_label,cell_type_alias_label,cell_type_designation_label,external_donor_name_label,specimen_type,outlier_call
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,AAACCCAAGGATTTCC-LKTX_190129_01_A01,AAACCCAAGGATTTCC-21L8TX_180927_001_A01,Inh L1-2 SST CCNJL,GABAergic,Sst,F,M1,all,CS1912131050,Inh L1-2 SST CCNJL,Neuron 50,H18.30.001,nucleus,False
2,AAACCCAAGTATGGCG-LKTX_190129_01_A01,AAACCCAAGTATGGCG-21L8TX_180927_001_A01,Exc L5-6 FEZF2 IFNG-AS1,Glutamatergic,L5/6 NP,F,M1,all,CS1912131116,Exc L5-6 FEZF2 IFNG-AS1,Neuron 116,H18.30.001,nucleus,False
3,AAACCCACAAAGTGTA-LKTX_190129_01_A01,AAACCCACAAAGTGTA-21L8TX_180927_001_A01,Exc L3-5 RORB LINC01202,Glutamatergic,L5 IT,F,M1,all,CS1912131087,Exc L3-5 RORB LINC01202,Neuron 87,H18.30.001,nucleus,False
4,AAACCCACACTACTTT-LKTX_190129_01_A01,AAACCCACACTACTTT-21L8TX_180927_001_A01,Exc L2 LINC00507 GLRA3,Glutamatergic,L2/3 IT,F,M1,all,CS1912131075,Exc L2 LINC00507 GLRA3,Neuron 75,H18.30.001,nucleus,False
5,AAACCCACAGTGAGCA-LKTX_190129_01_A01,AAACCCACAGTGAGCA-21L8TX_180927_001_A01,Oligo L2-6 OPALIN FTH1P3,Non-Neuronal,Oligo,F,M1,all,CS1912131120,Oligo L2-6 OPALIN FTH1P3,Non-neuron 3,H18.30.001,nucleus,False
6,AAACCCAGTCACCCTT-LKTX_190129_01_A01,AAACCCAGTCACCCTT-21L8TX_180927_001_A01,Exc L5-6 FEZF2 C9orf135-AS1,Glutamatergic,L6 CT,F,M1,all,CS1912131101,Exc L5-6 FEZF2 C9orf135-AS1,Neuron 101,H18.30.001,nucleus,False


In [19]:
#To show that these two columns provide no new information
print(sum(meta_data$region_label!="M1"))
print(sum(meta_data$specimen_type!="nucleus"))

[1] 0
[1] 0


In [10]:
meta_data <- select(meta_data, -region_label, -specimen_type)
head(meta_data)

,sample_name,exp_component_name,cluster_label,class_label,subclass_label,donor_sex_label,cortical_layer_label,cell_type_accession_label,cell_type_alias_label,cell_type_designation_label,external_donor_name_label,outlier_call
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,AAACCCAAGGATTTCC-LKTX_190129_01_A01,AAACCCAAGGATTTCC-21L8TX_180927_001_A01,Inh L1-2 SST CCNJL,GABAergic,Sst,F,all,CS1912131050,Inh L1-2 SST CCNJL,Neuron 50,H18.30.001,False
2,AAACCCAAGTATGGCG-LKTX_190129_01_A01,AAACCCAAGTATGGCG-21L8TX_180927_001_A01,Exc L5-6 FEZF2 IFNG-AS1,Glutamatergic,L5/6 NP,F,all,CS1912131116,Exc L5-6 FEZF2 IFNG-AS1,Neuron 116,H18.30.001,False
3,AAACCCACAAAGTGTA-LKTX_190129_01_A01,AAACCCACAAAGTGTA-21L8TX_180927_001_A01,Exc L3-5 RORB LINC01202,Glutamatergic,L5 IT,F,all,CS1912131087,Exc L3-5 RORB LINC01202,Neuron 87,H18.30.001,False
4,AAACCCACACTACTTT-LKTX_190129_01_A01,AAACCCACACTACTTT-21L8TX_180927_001_A01,Exc L2 LINC00507 GLRA3,Glutamatergic,L2/3 IT,F,all,CS1912131075,Exc L2 LINC00507 GLRA3,Neuron 75,H18.30.001,False
5,AAACCCACAGTGAGCA-LKTX_190129_01_A01,AAACCCACAGTGAGCA-21L8TX_180927_001_A01,Oligo L2-6 OPALIN FTH1P3,Non-Neuronal,Oligo,F,all,CS1912131120,Oligo L2-6 OPALIN FTH1P3,Non-neuron 3,H18.30.001,False
6,AAACCCAGTCACCCTT-LKTX_190129_01_A01,AAACCCAGTCACCCTT-21L8TX_180927_001_A01,Exc L5-6 FEZF2 C9orf135-AS1,Glutamatergic,L6 CT,F,all,CS1912131101,Exc L5-6 FEZF2 C9orf135-AS1,Neuron 101,H18.30.001,False


In [11]:
# To see the number of cell types, and the number of cells of each type
select(meta_data, class_label) %>%
    table()

class_label
    GABAergic Glutamatergic  Non-Neuronal 
        23992         48536          4005 

## 4. Explore and clean the counts matrix.

In [24]:
str(gene_matrix)

Classes ‘data.table’ and 'data.frame':	76533 obs. of  50282 variables:
 $ sample_name           : chr  "AAACCCAAGGATTTCC-LKTX_190129_01_A01" "AAACCCAAGTATGGCG-LKTX_190129_01_A01" "AAACCCACAAAGTGTA-LKTX_190129_01_A01" "AAACCCACACTACTTT-LKTX_190129_01_A01" ...
 $ DDX11L1               : int  0 0 0 0 0 0 0 0 0 0 ...
 $ WASH7P                : int  0 0 0 0 0 0 0 0 0 0 ...
 $ MIR6859-1             : int  0 0 0 0 0 0 0 0 0 0 ...
 $ MIR1302-2             : int  0 0 0 0 0 0 0 0 0 0 ...
 $ FAM138A               : int  0 0 0 0 0 0 0 0 0 0 ...
 $ LOC105379212          : int  0 0 0 0 0 0 0 0 0 0 ...
 $ OR4G4P                : int  0 0 0 0 0 0 0 0 0 0 ...
 $ OR4G11P               : int  0 0 0 0 0 0 0 0 0 0 ...
 $ OR4F5                 : int  0 0 0 0 0 0 0 0 0 0 ...
 $ LOC105379213          : int  0 0 0 0 0 0 0 0 0 0 ...
 $ CICP27                : int  0 0 0 0 0 0 0 0 0 0 ...
 $ LOC729737             : int  0 0 0 0 0 0 0 0 0 0 ...
 $ LOC100996442          : int  0 0 0 0 0 0 0 0 0 0 ...
 $ LOC1053792

In [28]:
head(gene_matrix)

sample_name,DDX11L1,WASH7P,MIR6859-1,MIR1302-2,FAM138A,LOC105379212,OR4G4P,OR4G11P,OR4F5,⋯,ND4,TRNH,TRNS2,TRNL2,ND5,ND6,TRNE,CYTB,TRNT,TRNP
<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
AAACCCAAGGATTTCC-LKTX_190129_01_A01,0,0,0,0,0,0,0,0,0,⋯,8,0,0,0,2,0,0,19,0,0
AAACCCAAGTATGGCG-LKTX_190129_01_A01,0,0,0,0,0,0,0,0,0,⋯,5,0,0,0,0,0,0,3,0,0
AAACCCACAAAGTGTA-LKTX_190129_01_A01,0,0,0,0,0,0,0,0,0,⋯,8,0,0,0,1,0,0,5,0,0
AAACCCACACTACTTT-LKTX_190129_01_A01,0,0,0,0,0,0,0,0,0,⋯,6,0,0,0,6,0,0,8,0,0
AAACCCACAGTGAGCA-LKTX_190129_01_A01,0,0,0,0,0,0,0,0,0,⋯,1,0,0,0,0,0,0,0,0,0
AAACCCAGTCACCCTT-LKTX_190129_01_A01,0,0,0,0,0,0,0,0,0,⋯,19,0,0,0,2,0,0,7,0,0


In [12]:
print(sum(duplicated(colnames(gene_matrix))))

[1] 0


## 5. Perform Exploratory Data Analysis (EDA).

In [13]:
meta_subset <- meta_data %>% 
    select(sample_name, cluster_label:subclass_label)
head(meta_subset)

,sample_name,cluster_label,class_label,subclass_label
,<chr>,<chr>,<chr>,<chr>
1,AAACCCAAGGATTTCC-LKTX_190129_01_A01,Inh L1-2 SST CCNJL,GABAergic,Sst
2,AAACCCAAGTATGGCG-LKTX_190129_01_A01,Exc L5-6 FEZF2 IFNG-AS1,Glutamatergic,L5/6 NP
3,AAACCCACAAAGTGTA-LKTX_190129_01_A01,Exc L3-5 RORB LINC01202,Glutamatergic,L5 IT
4,AAACCCACACTACTTT-LKTX_190129_01_A01,Exc L2 LINC00507 GLRA3,Glutamatergic,L2/3 IT
5,AAACCCACAGTGAGCA-LKTX_190129_01_A01,Oligo L2-6 OPALIN FTH1P3,Non-Neuronal,Oligo
6,AAACCCAGTCACCCTT-LKTX_190129_01_A01,Exc L5-6 FEZF2 C9orf135-AS1,Glutamatergic,L6 CT
